### Làm sạch tập dữ liệu sau khi crawl (mùa giải 2023-2024)


In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [2]:
match_df = pd.read_csv("../raw_data/raw_data_test.csv")
match_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,poss,attendance,captain,formation,referee,match report,notes,season,team,url
0,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,65.0,21572.0,Kevin De Bruyne,4-2-3-1,Craig Pawson,Match Report,NaN,2023-2024,Manchester City,https://fbref.com/en/squads/b8fd03ef/Mancheste...
1,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,59.0,53419.0,Kyle Walker,4-2-3-1,Robert Jones,Match Report,NaN,2023-2024,Manchester City,https://fbref.com/en/squads/b8fd03ef/Mancheste...
2,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,79.0,31336.0,Kyle Walker,4-2-3-1,Jarred Gillett,Match Report,NaN,2023-2024,Manchester City,https://fbref.com/en/squads/b8fd03ef/Mancheste...
3,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,68.0,52899.0,Kyle Walker,4-2-3-1,Michael Oliver,Match Report,NaN,2023-2024,Manchester City,https://fbref.com/en/squads/b8fd03ef/Mancheste...
4,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,68.0,62475.0,Kyle Walker,4-2-3-1,Andy Madley,Match Report,NaN,2023-2024,Manchester City,https://fbref.com/en/squads/b8fd03ef/Mancheste...


#### Số mẫu của tập test chưa đi tách đội


In [3]:
match_df.shape

(2280, 22)

In [4]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          2280 non-null   object 
 1   time          2280 non-null   object 
 2   comp          2280 non-null   object 
 3   round         2280 non-null   object 
 4   day           2280 non-null   object 
 5   venue         2280 non-null   object 
 6   result        2280 non-null   object 
 7   gf            2280 non-null   int64  
 8   ga            2280 non-null   int64  
 9   opponent      2280 non-null   object 
 10  xg            760 non-null    float64
 11  xga           760 non-null    float64
 12  poss          760 non-null    float64
 13  attendance    2272 non-null   float64
 14  captain       2280 non-null   object 
 15  formation     2280 non-null   object 
 16  referee       2280 non-null   object 
 17  match report  2280 non-null   object 
 18  notes         0 non-null    

### Clean data


1. Xóa các features không dùng cho việc build model


In [5]:
match_df = match_df.drop(
    columns=[
        "comp",
        "day",
        "match report",
        "poss",
        "captain",
        "notes",
        "url",
        "xg",
        "xga",
    ]
)
match_df.columns

Index(['date', 'time', 'round', 'venue', 'result', 'gf', 'ga', 'opponent',
       'attendance', 'formation', 'referee', 'season', 'team'],
      dtype='object')

2. Xử lý các giá trị NaN


In [6]:
match_df = match_df.dropna(subset=["result"])
match_df.shape

(2280, 13)

3. Xử lý formation bị lỗi dữ liệu


In [7]:
match_df["formation"].value_counts()

formation
4-2-3-1       720
4/3/2003      407
3/4/2003      180
4-3-3         172
4/4/2002      152
3-4-3          89
4-1-4-1        82
3/5/2002       80
4-4-1-1        71
4-4-2          54
3-5-2          41
5/3/2002       36
3-4-1-2        26
5/4/2001       22
3-2-4-1        21
4-3-1-2        20
4-1-2-1-2◆     16
4-3-2-1        16
4/5/2001       14
4-2-2-2        12
5-4-1          11
3-5-1-1         9
5-3-2           9
3-3-3-1         7
3-1-4-2         7
4-5-1           3
4-2-4-0         2
3-4-3◆          1
Name: count, dtype: int64

In [8]:
choices = ["4-2-3-1", "4-4-2", "4-3-3"]

# Fill NaN values with randomly chosen values from the list
match_df["formation"] = match_df["formation"].fillna(np.random.choice(choices))

match_df["formation"].value_counts()

formation
4-2-3-1       720
4/3/2003      407
3/4/2003      180
4-3-3         172
4/4/2002      152
3-4-3          89
4-1-4-1        82
3/5/2002       80
4-4-1-1        71
4-4-2          54
3-5-2          41
5/3/2002       36
3-4-1-2        26
5/4/2001       22
3-2-4-1        21
4-3-1-2        20
4-1-2-1-2◆     16
4-3-2-1        16
4/5/2001       14
4-2-2-2        12
5-4-1          11
3-5-1-1         9
5-3-2           9
3-3-3-1         7
3-1-4-2         7
4-5-1           3
4-2-4-0         2
3-4-3◆          1
Name: count, dtype: int64

In [9]:
# Xóa kí tự đặc biệt trong data cột 'formation'
match_df["formation"] = match_df["formation"].str.replace("◆", "", regex=False)


def nomalize_formation(value):
    if "/" in value:
        parts = value.split("/")
        parts[2] = parts[2][-1]
        return "-".join(parts)
    elif value.endswith("-0"):
        return value[:-2]
    else:
        return value


match_df["formation"] = match_df["formation"].apply(nomalize_formation)
match_df["formation"].unique()

array(['4-2-3-1', '4-3-3', '3-2-4-1', '3-4-3', '4-1-4-1', '4-4-2',
       '4-2-2-2', '4-4-1-1', '3-5-2', '5-3-2', '4-3-1-2', '4-1-2-1-2',
       '4-2-4', '4-3-2-1', '4-5-1', '5-4-1', '3-4-1-2', '3-5-1-1',
       '3-3-3-1', '3-1-4-2'], dtype=object)

4. Xử lý attendance bị missing data


In [10]:
# Fill giá trị thiếu trong cột 'attendance' bằng cách dùng group theo 2 đội đối đầu nhau rồi lấy trung bình
# vì tùy cặp đấu mà lượng khán giả sẽ cao
match_df["attendance"] = match_df.groupby(["team", "opponent"])["attendance"].transform(
    lambda x: x.fillna(x.mean())
)

# Vì có thể có các cặp chưa từng đấu nên sẽ còn giá trị thiếu => ta sẽ tiếp tục fill data theo trung bình đội đó
match_df["attendance"] = match_df.groupby("team")["attendance"].transform(
    lambda x: x.fillna(x.mean())
)
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        2280 non-null   object 
 1   time        2280 non-null   object 
 2   round       2280 non-null   object 
 3   venue       2280 non-null   object 
 4   result      2280 non-null   object 
 5   gf          2280 non-null   int64  
 6   ga          2280 non-null   int64  
 7   opponent    2280 non-null   object 
 8   attendance  2280 non-null   float64
 9   formation   2280 non-null   object 
 10  referee     2280 non-null   object 
 11  season      2280 non-null   object 
 12  team        2280 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 231.7+ KB


5. Điền giá trị mặc định cho cột time


In [11]:
match_df["time"] = match_df["time"].fillna("00:00")
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        2280 non-null   object 
 1   time        2280 non-null   object 
 2   round       2280 non-null   object 
 3   venue       2280 non-null   object 
 4   result      2280 non-null   object 
 5   gf          2280 non-null   int64  
 6   ga          2280 non-null   int64  
 7   opponent    2280 non-null   object 
 8   attendance  2280 non-null   float64
 9   formation   2280 non-null   object 
 10  referee     2280 non-null   object 
 11  season      2280 non-null   object 
 12  team        2280 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 231.7+ KB


6. Sửa lại tên đội đúng


In [12]:
# Hàm lấy ra dataFrame tên khác nhau của 2 cột
def compare_team_component():
    sorted_unique_teams = sorted(match_df["team"].unique())
    sorted_unique_opponents = sorted(match_df["opponent"].unique())

    # Tạo DataFrame để hiển thị các tên đội đã được sắp xếp
    team_df = pd.DataFrame(sorted_unique_teams, columns=["Team Names"])
    opponent_df = pd.DataFrame(sorted_unique_opponents, columns=["Opponent Names"])

    # Vì số lượng tên trong mỗi danh sách có thể không bằng nhau, ta cần thêm các hàng trống cho DataFrame ngắn hơn
    max_length = max(len(team_df), len(opponent_df))
    team_df = team_df.reindex(range(max_length)).fillna("")
    opponent_df = opponent_df.reindex(range(max_length)).fillna("")

    # Kết hợp hai DataFrame để dễ so sánh
    comparison_df = pd.concat([team_df, opponent_df], axis=1)
    comparison_df = comparison_df[
        comparison_df["Team Names"] != comparison_df["Opponent Names"]
    ]
    return comparison_df


comparison_df = compare_team_component()
comparison_df

,Team Names,Opponent Names
4,Brighton and Hove Albion,Brighton
15,Manchester United,Manchester Utd
16,Newcastle United,Newcastle Utd
18,Nottingham Forest,Nott'ham Forest
19,Sheffield United,Sheffield Utd
21,Tottenham Hotspur,Tottenham
23,West Ham United,West Ham
24,Wolverhampton Wanderers,Wolves


In [13]:
# Mảng các tên đội cần đổi lại để đồng nhất 2 cột
rename_dict = {
    "Brighton": "Brighton and Hove Albion",
    "Manchester Utd": "Manchester United",
    "Newcastle Utd": "Newcastle United",
    "Nott'ham Forest": "Nottingham Forest",
    "Sheffield Utd": "Sheffield United",
    "Tottenham": "Tottenham Hotspur",
    "West Brom": "West Bromwich Albion",
    "West Ham": "West Ham United",
    "Wolves": "Wolverhampton Wanderers",
    "Blackburn": "Blackburn Rovers",
    "Bolton": "Bolton Wanderers",
    "Charlton Ath": "Charlton Athletic",
    "Huddersfield": "Huddersfield Town",
    "QPR": "Queens Park Rangers",
}

match_df["opponent"] = match_df["opponent"].replace(rename_dict)

comparison_df = compare_team_component()
comparison_df

,Team Names,Opponent Names


### Check lại data lần cuối


In [14]:
match_df["season"].value_counts()

season
2023-2024    760
2022-2023    760
2021-2022    760
Name: count, dtype: int64

In [15]:
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d').strftime('%Y-%m-%d')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%Y-%m-%d')
        except ValueError:
            return pd.NaT

In [16]:
match_df["date"] = match_df["date"].apply(parse_date)

In [17]:
match_df.to_csv("../raw_data/clean_data_test.csv", index=False)